In [1]:
# データ処理のライブラリ
import pandas as pd
import numpy as np 
 
# データ可視化のライブラリ
import matplotlib.pyplot as plt
 
# 機械学習ライブラリ
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [3]:
# graphvizのインポート
import graphviz
 
# バージョンの確認
print(graphviz.__version__)

0.13.2


In [4]:
# CSVファイルの読み込み
df = pd.read_csv('usd_jpy_api.csv')

In [5]:
df.tail()

,time,close,open,high,low,volume
495,2018/07/17 06:00:00,112.908,112.320,112.933,112.231,19645
496,2018/07/18 06:00:00,112.866,112.914,113.144,112.723,17432
497,2018/07/19 06:00:00,112.501,112.887,113.187,112.072,26697
498,2018/07/20 06:00:00,111.474,112.504,112.631,111.401,33445
499,2018/07/23 06:00:00,111.164,111.420,111.523,110.760,16040


In [6]:
# 翌日終値 - 当日終値で差分を計算
df['close+1'] = df.close.shift(-1)
df['diff'] = df['close+1'] - df['close']
df = df[:-1]
 
# 最後の5行を確認
df.tail()

,time,close,open,high,low,volume,close+1,diff
494,2018/07/16 06:00:00,112.320,112.252,112.570,112.235,12518,112.908,0.588
495,2018/07/17 06:00:00,112.908,112.320,112.933,112.231,19645,112.866,-0.042
496,2018/07/18 06:00:00,112.866,112.914,113.144,112.723,17432,112.501,-0.365
497,2018/07/19 06:00:00,112.501,112.887,113.187,112.072,26697,111.474,-1.027
498,2018/07/20 06:00:00,111.474,112.504,112.631,111.401,33445,111.164,-0.310


In [7]:
# 上昇と下降のデータ割合を確認
m = len(df['close'])
print(len(df[(df['diff'] > 0)]) / m * 100)
print(len(df[(df['diff'] < 0)]) / m * 100)

50.70140280561122
49.298597194388776


In [8]:
# 上昇したら「1」、下降したら「0」へデータを変換
mask1 = df['diff'] > 0
mask2 = df['diff'] < 0
column_name = 'diff'
df.loc[mask1, column_name] = 1
df.loc[mask2, column_name] = 0
 
# 最初の5行を表示
df.head()

,time,close,open,high,low,volume,close+1,diff
0,2016/08/19 06:00:00,100.256,99.919,100.471,99.887,30965,100.335,1.0
1,2016/08/22 06:00:00,100.335,100.832,100.944,100.221,32920,100.253,0.0
2,2016/08/23 06:00:00,100.253,100.339,100.405,99.950,26069,100.460,1.0
3,2016/08/24 06:00:00,100.460,100.270,100.619,100.104,22340,100.546,1.0
4,2016/08/25 06:00:00,100.546,100.464,100.627,100.314,17224,101.876,1.0


In [9]:
# カラム名をtagertへ変換
df.rename(columns={"diff" : "target"}, inplace=True)
 
# 不要なカラムを削除
del df['time']
del df['close+1']
 
# カラムの並び替え
df = df[['target', 'close', 'open', 'high', 'low', 'volume']]
 
# 最初の5行を出力
df.head()

,target,close,open,high,low,volume
0,1.0,100.256,99.919,100.471,99.887,30965
1,0.0,100.335,100.832,100.944,100.221,32920
2,1.0,100.253,100.339,100.405,99.950,26069
3,1.0,100.460,100.270,100.619,100.104,22340
4,1.0,100.546,100.464,100.627,100.314,17224


In [11]:
# データセットの行数、列数を取得
n = df.shape[0]
p = df.shape[1]
 
# 訓練データとテストデータへ分割
train_start = 0
train_end = int(np.floor(0.8*n))
test_start = train_end + 1
test_end = n

In [40]:
print(train_start, train_end)
print(test_start, test_end)

0 399
400 499


In [51]:
data_train = df.loc[np.arange(train_start, train_end), :]
data_test = df.loc[np.arange(test_start, test_end), :]
 
# 訓練データとテストデータのサイズを確認
print(data_train.shape)
print(data_test.shape)

(399, 6)
(99, 6)


In [79]:
# 特徴量とターゲットにデータを切り分け
X_train = data_train.iloc[:,1]
y_train = data_train.iloc[:, 0]
X_test = data_test.iloc[:, 1]
y_test = data_test.iloc[:, 0]

In [82]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [83]:
# 決定技モデルの訓練
clf_2 = DecisionTreeClassifier(max_depth=2)
clf_2 = clf_2.fit(X_train, y_train)

ValueError: Expected 2D array, got 1D array instead:
array=[100.256 100.335 100.253 100.46  100.546 101.876 101.928 102.983 103.434
 103.246 103.992 103.461 102.034 101.752 102.5   102.745 101.86  102.569
 102.441 102.102 102.301 101.939 101.733 100.349 100.778 101.05  100.346
 100.453 100.705 101.064 101.403 101.665 102.919 103.513 103.969 102.953
 103.624 103.529 104.223 103.712 104.209 103.898 103.875 103.458 103.964
 103.856 104.195 104.235 104.49  105.299 104.758 104.838 104.172 103.322
 102.989 103.164 104.494 105.199 105.678 106.85  106.733 108.441 109.216
 109.095 110.165 110.961 110.873 111.164 112.547 113.378 113.303 111.946
 112.41  114.475 114.116 113.565 113.877 114.036 113.78  114.047 115.37
 115.039 115.2   117.06  118.202 117.963 117.117 117.892 117.575 117.564
 117.383 117.437 117.278 116.555 117.067 117.761 117.264 115.382 117.032
 116.073 115.788 115.426 114.736 114.548 114.222 112.625 114.646 114.877
 114.635 112.713 113.801 113.306 114.551 115.144 113.791 112.813 1